In [63]:
import sqlite3
import pandas as pd

In [91]:
conn = sqlite3.connect('test.db') # создаем базу данных
curs = conn.cursor() # открываем курсор - это некий объект базы данных, который позволяет выполнять SQL запросы
#curs.execute('''CREATE TABLE Highschooler(ID int, name varchar(100), grade int)''')
#curs.execute('''CREATE TABLE Friend(ID1 int, ID2 int)''')
#curs.execute('''CREATE TABLE Likes(ID1 int, ID2 int)''')

#curs.close()

In [92]:
# Посмотрели созданные таблицы
curs.execute('''
SELECT
    sql 
FROM 
    sqlite_master 
''')
for i in curs:
    print(i)

('CREATE TABLE Highschooler(ID int, name varchar(100), grade int)',)
('CREATE TABLE Friend(ID1 int, ID2 int)',)
('CREATE TABLE Likes(ID1 int, ID2 int)',)


In [93]:
##Заполняем наши таблицы данными
curs.executescript('''
insert into Highschooler values (1510,'Jordan',9);
insert into Highschooler values (1689,'Gabriel',9);
insert into Highschooler values (1381,'Tiffany',9);
insert into Highschooler values (1709,'Cassandra',9);
insert into Highschooler values (1101,'Haley',10);
insert into Highschooler values (1782,'Andrew',10);
insert into Highschooler values (1468,'Kris',10);
insert into Highschooler values (1641,'Brittany',10);
insert into Highschooler values (1247,'Alexis',11);
insert into Highschooler values (1316,'Austin',11);
insert into Highschooler values (1911,'Gabriel',11);
insert into Highschooler values (1501,'Jessica',11);
insert into Highschooler values (1304,'Jordan',12);
insert into Highschooler values (1025,'John',12);
insert into Highschooler values (1934,'Kyle',12);
insert into Highschooler values (1661,'Logan',12);
''')

curs.executescript('''
insert into Friend values (1510, 1381);
insert into Friend values (1510, 1689);
insert into Friend values (1689, 1709);
insert into Friend values (1381, 1247);
insert into Friend values (1709, 1247);
insert into Friend values (1689, 1782);
insert into Friend values (1782, 1468);
insert into Friend values (1782, 1316);
insert into Friend values (1782, 1304);
insert into Friend values (1468, 1101);
insert into Friend values (1468, 1641);
insert into Friend values (1101, 1641);
insert into Friend values (1247, 1911);
insert into Friend values (1247, 1501);
insert into Friend values (1911, 1501);
insert into Friend values (1501, 1934);
insert into Friend values (1316, 1934);
insert into Friend values (1934, 1304);
insert into Friend values (1304, 1661);
insert into Friend values (1661, 1025);
''')

curs.executescript('''
insert into Likes values(1689, 1709);
insert into Likes values(1709, 1689);
insert into Likes values(1782, 1709);
insert into Likes values(1911, 1247);
insert into Likes values(1247, 1468);
insert into Likes values(1641, 1468);
insert into Likes values(1316, 1304);
insert into Likes values(1501, 1934);
insert into Likes values(1934, 1501);
insert into Likes values(1025, 1101);
''')


4. For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order.

In [94]:
students_like_each_other = pd.read_sql("""SELECT table_1.name, table_1.grade, table_2.name, table_2.grade 
FROM 
((SELECT ID1, name, grade, ID2 FROM highschooler, likes WHERE ID = ID1) as table_1
JOIN
(SELECT ID1, name, grade, ID2 FROM highschooler, likes WHERE ID = ID1) as table_2
ON table_1.ID1 = table_2.ID2) 
GROUP BY 
table_1.name, table_2.name, table_1.grade, 
table_2.grade
HAVING 
table_1.id2 = table_2.id1 
""", my_DB)
print(students_like_each_other)

        name  grade       name  grade
0  Cassandra      9    Gabriel      9
1    Gabriel      9  Cassandra      9
2    Jessica     11       Kyle     12
3       Kyle     12    Jessica     11


5. Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade

In [95]:
my_DB = sqlite3.connect('test.db')
student_not_likes = pd.read_sql("""SELECT DISTINCT
Highschooler.ID,Highschooler.name,Highschooler.grade
FROM
Highschooler
WHERE
Highschooler.ID NOT IN (SELECT Likes.ID1 FROM Likes)
AND
Highschooler.ID NOT IN (SELECT Likes.ID2 FROM Likes)
ORDER BY Highschooler.grade DESC,Highschooler.name ASC
""", my_DB)#делаем запрос по отсутсвиям лайков и сортируем по возрастанию класса, и в алфавитном порядке внутри name
print(student_not_likes)

     ID     name  grade
0  1661    Logan     12
1  1510   Jordan      9
2  1381  Tiffany      9


6. who like themselves

In [96]:
my_DB = sqlite3.connect('test.db')
who_likes_themselves = pd.read_sql("""SELECT DISTINCT
Highschooler.ID,Highschooler.name
FROM
Highschooler
WHERE
Highschooler.ID IN (SELECT Likes.ID1 FROM Likes WHERE Likes.ID1=Likes.ID2)
AND
Highschooler.ID IN (SELECT Likes.ID2 FROM Likes WHERE Likes.ID1=Likes.ID2)

""", my_DB)#делаем запрос кто лайкнул сам себя т.е. Likes.ID1 = Likes.ID2
print(who_likes_themselves)
#как видим таких нет

Empty DataFrame
Columns: [ID, name]
Index: []


### у нас есть 3 таблицы:
#### - Старшеклассник ( Highschooler ) со столбцами id (идентификатор ), name (имя), grade (класс)
#### - Друг (Friend) со столбцами (id1) старшеклассник,  (id2) друг старшеклассника
#### - Лайки (Likes) со столбцами (id1) кто лайкнул, (id2) кого лайкнули

### Выполним следующие задания:
1. Find person, who likes more than anyone 
2. Find the names of all students who are friends with someone named Gabriel
3. For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like.
4. For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order.
5. Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade
6. who like themselves
7. For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades. 
8. Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade.
9. For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C. 
10. Find the difference between the number of students in the school and the number of different first names. 
11. What is the average number of friends per student? (Your result should be just one number.) 
12. Find the name and grade of all students who are liked by more than one other student. 
13. For every situation where student A likes student B, but student B likes a different student C, return the names and grades of A, B, and C.
14. Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count Cassandra, even though technically she is a friend of a friend. 
15. It's time for the seniors to graduate. Remove all 12th graders from Highschooler.
16. If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple.
17. For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself.